In [1]:
import torch
from multipliers import *
import timeit
import time
import os

In [2]:
# torch.set_num_threads(1)
# torch.set_num_interop_threads(1)

# os.environ["OMP_NUM_THREADS"] = '1'
# os.environ["MKL_NUM_THREADS"] = '1'
# os.environ["NUMEXPR_NUM_THREADS"] = '1'
# os.environ["OPENBLAS_NUM_THREADS"] = '1'
# os.environ["VECLIB_MAXIMUM_THREADS"] = '1'
# os.environ["CUDA_VISIBLE_DEVICES"] = ''

# torch.cuda.is_available()

In [2]:
DTYPE = torch.bfloat16

def generate_random_int_vector(size, low=0, high=100):
    return torch.randint(low, high, size).to(dtype=DTYPE)
    
def generate_random_ternary_matrix(n, m):
    return torch.randint(-1, 2, (n, m)).to(dtype=DTYPE)    

In [8]:
matrix = generate_random_ternary_matrix(4096, 1024)
vector = generate_random_int_vector((6, 4096))

In [9]:
for k in [2, 3, 4, 5, 6]:
    print(f"k: {k}")
    
    mult = RSRTernaryMultiplier(matrix, k=k)
    
    print("Timing...")

    %timeit mult.multiply(vector)
    %timeit NaiveMultiplier(matrix).multiply(vector)

k: 2
Timing...
311 ms ± 3.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
828 ms ± 27.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
k: 3
Timing...


KeyboardInterrupt: 

In [3]:
matrix = generate_random_ternary_matrix(2**12, 2**12 + 1)
vector = torch.randn((3, 2**12), dtype=DTYPE)

mult = RSRTernaryMultiplier(matrix, k=2)
a = mult.multiply(vector)
b = NaiveMultiplier(matrix).multiply(vector)

a.shape, b.shape, a, b

(torch.Size([3, 4097]),
 torch.Size([3, 4097]),
 tensor([[  31.6250,   33.5000,   62.5000,  ..., -102.0000,   27.7500,
           -24.2500],
         [ -50.0000,  -60.0000,   25.7500,  ...,   97.5000,   57.5000,
          -121.0000],
         [  40.5000,  -76.0000,   22.6250,  ...,    3.6250,  -96.0000,
            88.0000]], dtype=torch.bfloat16),
 tensor([[  31.6250,   33.5000,   62.5000,  ..., -102.0000,   27.6250,
           -24.1250],
         [ -50.0000,  -60.0000,   25.7500,  ...,   97.5000,   57.2500,
          -120.5000],
         [  40.2500,  -76.0000,   22.6250,  ...,    3.7188,  -96.0000,
            88.0000]], dtype=torch.bfloat16))

### Opt Results